In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [3]:
df = pd.read_csv('/Users/bumsoojoe/Desktop/kaggel project/Data/train_data.csv')

In [4]:
df

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,5326,1.933,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,4647,2.347,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,102843,11.824,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,7665,3.519,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643675,7951fb2f47f1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,37530,7.248,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643676,e0dc5823e5e1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,337248,17.902,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643677,0d6036529b42,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,44053,6.700,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643678,46d1f07d723b,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,108600,11.716,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# 하나의 서열이 같은 것이 존재하는가?
test1 = df['sequence'][0]
test2 = df['sequence'][0]
s = 0
n = 0
for i in range(0,len(test1)):
    if test1[i] == test2[i]:
        s += 1
    else:
        n += 1
print('같은 염기서열 수 :',s)
print('다른 염시서열 수 :',n)

같은 염기서열 수 : 170
다른 염시서열 수 : 0


In [14]:
df['experiment_type'].value_counts()

experiment_type
2A3_MaP    821840
DMS_MaP    821840
Name: count, dtype: int64

In [29]:
# 같은 서열이 과연 몇개나 존재하는가?
test3 = list(df[df['experiment_type']=='2A3_MaP']['sequence'].value_counts().index)
print(len(list(df[df['experiment_type']=='2A3_MaP']['sequence'])) - len(test3))

15267


In [32]:
len(test3)

806573

In [30]:
test4 = list(df[df['experiment_type']=='DMS_MaP']['sequence'].value_counts().index)
print(len(list(df[df['experiment_type']=='DMS_MaP']['sequence'])) - len(test3))

15267


In [34]:
print(len(df[df['experiment_type']=='DMS_MaP']['dataset_name'].value_counts().index))

23


In [49]:
first_row = df[0:1]

# NaN이 아닌 컬럼만 반환
non_nan_columns = first_row.dropna(axis=1)

non_nan_columns

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0027,reactivity_0028,reactivity_0029,...,reactivity_error_0120,reactivity_error_0121,reactivity_error_0122,reactivity_error_0123,reactivity_error_0124,reactivity_error_0125,reactivity_error_0126,reactivity_error_0127,reactivity_error_0128,reactivity_error_0129
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,-0.039,0.455,-0.207,...,0.224,0.085,0.085,0.1,0.224,0.286,0.246,0.085,0.224,0.254


In [46]:
list(df[0:1].columns)[7:]

['reactivity_0001',
 'reactivity_0002',
 'reactivity_0003',
 'reactivity_0004',
 'reactivity_0005',
 'reactivity_0006',
 'reactivity_0007',
 'reactivity_0008',
 'reactivity_0009',
 'reactivity_0010',
 'reactivity_0011',
 'reactivity_0012',
 'reactivity_0013',
 'reactivity_0014',
 'reactivity_0015',
 'reactivity_0016',
 'reactivity_0017',
 'reactivity_0018',
 'reactivity_0019',
 'reactivity_0020',
 'reactivity_0021',
 'reactivity_0022',
 'reactivity_0023',
 'reactivity_0024',
 'reactivity_0025',
 'reactivity_0026',
 'reactivity_0027',
 'reactivity_0028',
 'reactivity_0029',
 'reactivity_0030',
 'reactivity_0031',
 'reactivity_0032',
 'reactivity_0033',
 'reactivity_0034',
 'reactivity_0035',
 'reactivity_0036',
 'reactivity_0037',
 'reactivity_0038',
 'reactivity_0039',
 'reactivity_0040',
 'reactivity_0041',
 'reactivity_0042',
 'reactivity_0043',
 'reactivity_0044',
 'reactivity_0045',
 'reactivity_0046',
 'reactivity_0047',
 'reactivity_0048',
 'reactivity_0049',
 'reactivity_0050',
